In [ ]:
# задать конфигурацию retriever-части

In [ ]:
# загрузить benchmark-датасет
# инифицализировать класс с метриками

In [ ]:
# инициализировать ретривер

In [ ]:
# для каждого запроса получить список релевантных чанков

In [ ]:
# посчитать метрики 
# сохранить результат